In [37]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import pytz
from datetime import timezone,datetime,timedelta
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import gc

import api.util
#from api.data_collector import DataCollector
from api.sofa_dp import SofaDataProvider
from api.op_dp import OpDataProvider

from IPython.display import display
pd.options.display.max_columns = None
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
class DataCollector:
    def __init__(self):
        self.LOCAL_TZ = 'Asia/Almaty'
        self.SERVER_TZ = 'UTC'
        self.DATA_PATH='data/'
        self.ELO_DATA_PATH='data/elo/'
        self.PREREQUISITES_PATH='prerequisites/'
        self.COL_CAT=[]
        self.COL_NUM=[]
        self.COL_LBL=[]
        self.COL_INF=[]
    
    def _load_prerequisites(self,name):
        with open(os.path.join(self.PREREQUISITES_PATH, name),'rb') as f:
            encoder = pickle.load(f)
        return encoder
    
    def _save_prerequisite(self, name, data):
        folder='prerequisites/'
        os.makedirs(self.PREREQUISITES_PATH, mode=0o777, exist_ok=True)
        with open(os.path.join(self.PREREQUISITES_PATH, name), mode='wb') as f:
            pickle.dump(data, f) 

    def _ff(self, columns):
        if len(self.INCLUDE)>0:
            return [x for x in columns if x in self.INCLUDE]
        else:
            return [x for x in columns if x not in self.EXCLUDE]
    
    def _encode(self, enctype, features, outs, df):
        if (len(self.INCLUDE)>0 and outs[0] in self.INCLUDE) or outs[0] in self.EXCLUDE:
            return df
        name='_'.join(features)
        if self.LOAD:
            encoder=self._load_prerequisites(f'{enctype}_{features[0]}')
        else:
            if enctype=='sc':
                encoder = MinMaxScaler()
            elif enctype=='le':
                encoder = LabelEncoder()
            elif enctype=='ohe':
                encoder = OneHotEncoder()
            if len(features)==1:
                encoder.fit(df[features].values)
            else:
                encoder.fit(pd.concat([pd.DataFrame(df[features[0]].unique(), columns=[name]),pd.DataFrame(df[features[1]].unique(), columns=[name])])[name])
            self._save_prerequisite(f'{enctype}_{name}', encoder)
        if  enctype=='ohe':
            return encoder.transform(df[features].values).toarray()
        if len(features)==1:
            df[outs[0]] = encoder.transform(df[features].values)
        else:
            df[outs[0]] = encoder.transform(df[features[0]])
            df[outs[1]] = encoder.transform(df[features[1]])
        return df

    def _encode_teams(self, df):
        teams_name=self.ELO_DATA_PATH+'teams.csv'
        teams_saved=pd.read_csv(teams_name, index_col=None)
        teams=df[['team']].dropna().drop_duplicates()
        teams_new=teams[~teams.team.isin(teams_saved.team)]
        print(teams_new)
        if not teams_new.empty:
            print('New teams!')
            id=teams_saved.id.max()+1
            #id=0
            teams_list=[]
            for row in teams_new.itertuples():
                if len(row.team)>1:
                    teams_list.append({'team':row.team, 'id':id})
                    id+=1
                    #break
            teams_saved=pd.concat([teams_saved,pd.DataFrame(teams_list)])
            teams_saved.id=teams_saved.id.astype(int)
            teams_saved.to_csv(teams_name, index=False)
        df=df.merge(teams_saved, on='team', how='left')
        return df
    
    def _add_elo(self, df_src,df_elo):
        df_teams=pd.read_csv(self.DATA_PATH+'teams.csv', index_col=None)
        df_elo_merged=df_elo.merge(df_teams[['id','tid']], on='id', how='left').drop_duplicates()
        df_elo_merged=df_elo_merged.dropna()
        df_src['de']=df_src.ds.apply(lambda x: x.strftime('%Y-%m-%d'))
        df_elo_merged=df_elo_merged.rename(columns={'tid':'tid1', 'elo':'elo1'})
        df_src=df_src.merge(df_elo_merged[['tid1','de','elo1']], on=['tid1','de'], how='left')
        df_elo_merged=df_elo_merged.rename(columns={'tid1':'tid2', 'elo1':'elo2'})
        df_src=df_src.merge(df_elo_merged[['tid2','de','elo2']], on=['tid2','de'], how='left')
        return df_src

    def _provide_elo(self):
        df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(self.DATA_PATH+'elo/', 'elo_*.csv'))))
        df=df[['Club', 'Country', 'Level', 'Elo', 'From', 'To']]
        df.columns=['team', 'country', 'level', 'elo', 'ds', 'de']
        df=self._encode_teams(df)
        return df

    def _provide_sofa(self):
        dp=SofaDataProvider(load=True)
        df=dp._load_data()
        return df.drop_duplicates(subset='mid', keep='last')

    def _provide_op(self):
        dp=OpDataProvider(load=True)
        df=dp._load_data()
        return df

    def _bind_sofa_op(self,df):
        df_op=self._provide_op()
        df_binds=pd.read_csv('data/binds_ss_op.csv', index_col=None)
        df_op=df_op.merge(df_binds[['op_mid','mid']], left_on='mid', right_on='op_mid')
        return df.merge(df_op[['mid_y','odds_away','odds_draw','odds_home','oddsprob_home','oddsprob_draw','oddsprob_away','drift_home','drift_away','drift_draw']], left_on='mid', right_on='mid_y', how='left')

    def _load_data(self):
        #df_op=self._provide_sofa()
        df_sofa=self._provide_sofa()
        df_elo=self._provide_elo()
        df_sofa=self._add_elo(df_sofa,df_elo)
        df_sofa=self._bind_sofa_op(df_sofa)
        return df_sofa
    
    def provide_data(self, double=True):
        df=self._load_data()
        df['psft']=df.sc1-df.sc2
        df['psht']=df.homeScoreHT-df.awayScoreHT
        df['w1']=np.where(df.winner=='home',1,0)
        df['wx']=np.where(df.winner=='draw',1,0)
        df['w2']=np.where(df.winner=='away',1,0)
        df_home=df.copy()
        df_home=df_home.rename(columns={'homeScoreHT':'ht1','awayScoreHT':'ht2','sc1':'ft1','sc2':'ft2','vote_home':'vote1','vote_draw':'votex','vote_away':'vote2','home_formation':'form1','away_formation':'form2'})
        if double:
            df_home['side']=1
            df_away=df.copy()
            df_away['side']=0
            df_away=df_away.rename(columns={'homeScoreHT':'ht2','awayScoreHT':'ht1','sc1':'ft2','sc2':'ft1','vote_home':'vote2','vote_draw':'votex','vote_away':'vote1',
                                            'home_formation':'form2','away_formation':'form1','w1':'w2','w2':'w1','elo1':'elo2','elo2':'elo1','t1':'t2','t2':'t1',
                                            'tid1':'tid2','tid2':'tid1','odds_away':'odds_home','odds_home':'odds_away','oddsprob_home':'oddsprob_away',
                                            'oddsprob_away':'oddsprob_home','drift_home':'drift_away','drift_away':'drift_home',
                                            'possession1':'possession2', 'shont1':'shont2', 'shofft1':'shofft2', 'corners1':'corners2', 
                                            'offsides1':'offsides2', 'fouls1':'fouls2', 'cards1':'cards2', 'gksaves1':'gksaves2',
                                            'possession2':'possession1', 'shont2':'shont1', 'shofft2':'shofft1', 'corners2':'corners1', 
                                            'offsides2':'offsides1', 'fouls2':'fouls1', 'cards2':'cards1', 'gksaves2':'gksaves1'})
            df_away['psft']=df_away['psft']*-1
            df_away['psht']=df_away['psht']*-1

            df_home=pd.concat([df_home,df_away], axis=0)

        return df_home.reset_index(drop=True)

dp=DataCollector()
dp._provide_op()

In [32]:
dp=DataCollector()
#df=dp.provide_data(double=False)
df=dp.provide_data()

Empty DataFrame
Columns: [team]
Index: []


df

In [33]:
#['side', 'country', 'country_id', 'liga', 'mid', 'round', 'ds', 'de', 'tid1', 'tid2', 't1', 't2', 'w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft', 
#       'winner', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2']
COL_CUR=['country_id', 'round', 'ds', 'de', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2','oddsprob_home','oddsprob_draw','oddsprob_away','drift_home','drift_away','drift_draw']
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'psht', 'psft', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']
COL_CAT=['country_id','form1', 'form2', 'pop_r']
COL_BIN=['side']
COL_NUM=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'psht', 'psft', 'vote1', 'votex', 'vote2', 'elo1', 'elo2','oddsprob_home','oddsprob_draw','oddsprob_away','drift_home','drift_away','drift_draw', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']
COL_INF=['side','country', 'liga', 'mid', 'round', 'ds', 't1', 't2','tid1', 'tid2', 'w1', 'wx', 'w2',  'ft1', 'ft2','winner','odds_away','odds_draw','odds_home']

#df=pd.read_csv('data/matches.csv', index_col=None)
#df['elo1'].fillna((df['elo1'].mean()), inplace=True)
#df['elo2'].fillna((df['elo2'].mean()), inplace=True)


In [34]:
def calc_stat(df_, group_feature, name, cols):
    df_=df_.sort_values(by='ds')
    cols_out=['ds','mid','tid']
    df_['C']=df_.groupby([group_feature])['N'].apply(lambda x : x.shift().cumsum())#.cumsum()#-1
    #display(df_)
    for col in cols:
        print(col)
        cols_out.append(f'{col}_{name}_avg')
        df_[f'{col}_{name}_sum']=df_.groupby([group_feature])[col].apply(lambda x : x.shift().cumsum())#.cumsum()#-df_teams[col]
        df_[f'{col}_{name}_avg']=df_[f'{col}_{name}_sum']/df_['C']
        df_.drop(columns=[f'{col}_{name}_sum'], inplace=True)
        #cols_out.append(f'{col}_{name}_sum')
    df_=df_.rename(columns={group_feature:'tid'})
    return df_[cols_out]

def calc_stat_n(df_, group_feature, name, cols, n):
    df_=df_.sort_values(by='ds')
    cols_out=['ds','mid','tid']
    df_['C']=df_.groupby([group_feature])['N'].apply(lambda x : x.shift().rolling(min_periods=1, window=n).sum())
    #display(df_)
    for col in cols:
        print(col)
        cols_out.append(f'{col}_{name}_{n}')
        df_[f'{col}_{name}_sum']=df_.groupby([group_feature])[col].apply(lambda x : x.shift().rolling(min_periods=1, window=n).sum())
        df_[f'{col}_{name}_{n}']=df_[f'{col}_{name}_sum']/df_['C']
        df_.drop(columns=[f'{col}_{name}_sum'], inplace=True)
        #cols_out.append(f'{col}_{name}_sum')
    df_=df_.rename(columns={group_feature:'tid'})
    return df_[cols_out]

In [35]:
deep=3
df['N']=1
cols_cum=COL_NUM
df_all=calc_stat(df, 'tid1', 'tt',cols_cum).sort_values(by=['mid','tid'])
df_all3=calc_stat_n(df, 'tid1', 'tt',cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_all.columns[3:]]
res=df_all3[df_all3.columns[3:]].values-df_all[df_all.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_all=pd.concat([df_all,df_form], axis=1)

df_home=calc_stat(df[df['side']==1], 'tid1', 'ts', cols_cum).sort_values(by=['mid','tid'])
df_home3=calc_stat_n(df[df['side']==1], 'tid1', 'ts', cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_home.columns[3:]]
res=df_home3[df_home3.columns[3:]].values-df_home[df_home.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_home=pd.concat([df_home,df_form], axis=1)

df_away=calc_stat(df[df['side']==0], 'tid1', 'ts', cols_cum).reset_index(drop=True).sort_values(by=['mid','tid'])
df_away3=calc_stat_n(df[df['side']==0], 'tid1', 'ts', cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_away.columns[3:]]
res=df_away3[df_away3.columns[3:]].values-df_away[df_away.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_away=pd.concat([df_away,df_form], axis=1)

#df_home=calc_stat(df, 'tid1', 'th', cols_cum)
#df_away=calc_stat(df, 'tid2', 'ta', cols_cum)

df_all.to_csv('data/teams_stats_all.csv', index=False)
df_home.to_csv('data/teams_stats_home.csv', index=False)
df_away.to_csv('data/teams_stats_away.csv', index=False)
df_all3.to_csv('data/teams_stats_all3.csv', index=False)
df_home3.to_csv('data/teams_stats_home3.csv', index=False)
df_away3.to_csv('data/teams_stats_away3.csv', index=False)

w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob_home
oddsprob_draw
oddsprob_away
drift_home
drift_away
drift_draw
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob_home
oddsprob_draw
oddsprob_away
drift_home
drift_away
drift_draw
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob_home
oddsprob_draw
oddsprob_away
drift_home
drift_away
drift_draw
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
o

df_all=pd.read_csv('data/teams_stats_all.csv', index_col=None)
df_home=pd.read_csv('data/teams_stats_home.csv', index_col=None)
df_away=pd.read_csv('data/teams_stats_away.csv', index_col=None)

df_all=pd.read_csv('data/teams_stats_all.csv', index_col=None)
df_home=pd.read_csv('data/teams_stats_home.csv', index_col=None)
df_away=pd.read_csv('data/teams_stats_away.csv', index_col=None)

df_all.columns

id=9270007
tid1=594
tid2=1499
display(df_all3[df_all3.mid==id])
display(df_home3[df_home3.mid==id])
display(df_away3[df_away3.mid==id])


### df_all
- `tt_avg` - Все матчи до текущего, среднее
- `tt_form` - Среднее по 3 последним, без `tt_avg`

### df_home
Статсы только тех игр, когда команда играла дома
- `ts_avg` - Все матчи до текущего, среднее
- `ts_form` - Среднее по 3 последним, без `ts_avg`

### df_away
Статсы только тех игр, когда команда играла не дома
- `ts_avg` - Все матчи до текущего, среднее
- `ts_form` - Среднее по 3 последним, без `ts_avg`

Все, что 1 относится к самой команде, а с 2 - к её оппонентам, объединив данные добавлено два префикса - `tar_` и `_opp` соответственно.

Нужно скопировать статсы по командам с учетом:
1. Таргетная команда и оппонент.
2. Дома или нет.

## Вычисление diffs
Что есть 1 и 2 для разных статсов? 

На примере w1 - wx - w2:
- tar_w1_tt_avg - среднее побед таргетной команды по всем играм
- tar_wx_tt_avg - среднее ничей таргетной команды по всем играм
- tar_w2_tt_avg - среднее проигрышей таргетной команды по всем играм
- opp_w1_tt_avg - среднее побед оппонента по всем играм
- opp_wx_tt_avg - среднее ничей оппонента по всем играм
- opp_w2_tt_avg - среднее проигрышей оппонента по всем играм

tar_w1_tt_form - разница между последними играми и всей статистикой, может быть как положительной, если в последние игры параметр больше, так и отрицательной

аналогично с ts, только учитывается статистика только по играм дома либо в гостях

## Перекрестные сравнения: забил - пропустил
- tar_ft1_tt_avg - среднее количество голов забила таргетная команда по всем играм (оказала давление - graph1)
- tar_ft2_tt_avg - среднее количество голов пропустила таргетная команда по всем играм (позволила давление - graph2)
- opp_ft1_tt_avg - среднее количество голов забила команда оппонента по всем играм
- opp_ft2_tt_avg - среднее количество голов пропустила команда оппонента по всем играм

tar_ft1_tt_avg и opp_ft2_tt_avg => `ft1_tt_avg`, tar_ft2_tt_avg и opp_ft1_tt_avg => `ft2_tt_avg`, а также ft1_tt_avg - ft2_tt_avg => `diff_ft_tt_avg`

tar_graph1_tt_avg и opp_graph2_tt_avg => `graph1_tt_avg`, tar_graph2_tt_avg и opp_graph1_tt_avg => `graph2_tt_avg`, а также graph1_tt_avg - graph2_tt_avg => `diff_graph_tt_avg`

tar_w1_tt_avg и opp_w2_tt_avg => `w1_tt_avg`, tar_w2_tt_avg и opp_w1_tt_avg => `w2_tt_avg`, tar_wx_tt_avg и opp_wx_tt_avg => `wx_tt_avg`, а также w1_tt_avg - w2_tt_avg => `diff_w_tt_avg`

tar_psft_tt_avg и opp_psft_tt_avg => `psft_tt_avg`

In [38]:
# Копируем данные всех игр, и домашних и нет, вперемешку. Это - таргетный датасет.
# Префикс для него - tar_, а суффикс - _tt_.
df_all_target=df_all.copy()
df_all_target.columns=np.hstack([df_all.columns[:3],['tar_'+x for x in df_all.columns[3:]]])

# Точно такой же датасет называем оппонентным.
# Префикс для него - opp_, а суффикс - _tt_.
df_all_opponent=df_all.copy()
df_all_opponent.columns=np.hstack([df_all.columns[:3],['opp_'+x for x in df_all.columns[3:]]])

# Для команд, игравших дома делаем датасет из домашнего (домашний таргетный). 
# Префикс для него - tar_, а суффикс - _ts_.
df_home_target=df_home.copy()
df_home_target.columns=np.hstack([df_home.columns[:3],['tar_'+x.replace('_th_','_ts_') for x in df_home.columns[3:]]])
# Для их противников делаем датасет из гостевого (гостевой оппонентный). 
# Префикс для него - opp_, а суффикс - _ts_.
df_away_opponent=df_away.copy()
df_away_opponent.columns=np.hstack([df_away.columns[:3],['opp_'+x.replace('_ta_','_ts_') for x in df_away.columns[3:]]])

# Для команд, игравших в гостях делаем датасет из гостевого (гостевой таргетный). 
# Префикс для него - tar_, а суффикс - _ts_.
df_away_target=df_away.copy()
df_away_target.columns=np.hstack([df_away.columns[:3],['tar_'+x.replace('_ta_','_ts_') for x in df_away.columns[3:]]])
# Для их противников делаем датасет из домашнего (домашний оппонентный). 
# Префикс для него - opp_, а суффикс - _ts_.
df_home_opponent=df_home.copy()
df_home_opponent.columns=np.hstack([df_home.columns[:3],['opp_'+x.replace('_th_','_ts_') for x in df_home.columns[3:]]])

# Итого есть 6 датасетов с шаблонными названиями переменных:
# tar_???_tt_, opp_???_tt_ - для статсов по всем играм
# tar_???_ts_, opp_???_ts_ - для статсов с учётом дома / в гостях для таргетных команд, игравших дома
# tar_???_ts_, opp_???_ts_ - для статсов с учётом дома / в гостях для таргетных команд, игравших в гостях 

# Делаем два датасета для таргетных домашних и гостевых команд.
# В данные берём лишь информационные поля (по которым потом проводить анализ) - COL_INF, 
# и текущие данные, доступные к сбору перед матчем - COL_CUR. 
df_1=df[df['side']==1][COL_INF+COL_CUR]
df_2=df[df['side']==0][COL_INF+COL_CUR]

# Заполняем данные для таргетных команд, играющих дома:
# Статсы по всем данным для первой команды (таргет)                     tar_???_tt_
df_1=df_1.merge(df_all_target[df_all_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем данным для второй команды (оппонент)                   opp_???_tt_
df_1=df_1.merge(df_all_opponent[df_all_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')
# Статсы по всем играм дома для первой команды (таргет домашняя)        tar_???_ts_
df_1=df_1.merge(df_home_target[df_home_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем играм в гостях для второй команды (оппонент гостевая)  opp_???_ts_
df_1=df_1.merge(df_away_opponent[df_away_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')

# То же для таргетных команд, играющих в гостях:
# Статсы по всем данным для первой команды (таргет)                     tar_???_tt_
df_2=df_2.merge(df_all_target[df_all_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем данным для второй команды (оппонент)                   opp_???_tt_
df_2=df_2.merge(df_all_opponent[df_all_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')
# Статсы по всем играм в гостях для первой команды (таргет гостевая)    tar_???_ts_
df_2=df_2.merge(df_away_target[df_away_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем играм дома для второй команды (оппонент домашняя)      opp_???_ts_
df_2=df_2.merge(df_home_opponent[df_home_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')

# Объединяем оба датасета и удаляем дубликаты колонок:
df_=pd.concat([df_1,df_2], axis=0)
df_.drop(columns=['tid_x','tid_y'], inplace=True)

# Очищаем память
del df_all_target,df_all_opponent,df_home_target,df_away_opponent,df_away_target,df_home_opponent,df_1,df_2
gc.collect()

208

# Diffs
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']

cols_tar=[x for x in df_.columns if 'tar_' in x]
cols_opp=[x for x in df_.columns if 'opp_' in x]
cols_diff=[x.replace('tar_','_diff_') for x in cols_tar]
df_.reset_index(drop=True, inplace=True)
df_=pd.concat([df_,pd.DataFrame(df_[cols_tar].values-df_[cols_opp].values, columns=cols_diff)], axis=1)
df_['diff_vote12']=df_['vote1']-df_['vote2']
df_['diff_elo']=df_['elo1']-df_['elo2']
df_['diff_op']=df_['oddsprob_home']-df_['oddsprob_away']
df_.to_csv('data/stats_generated1.csv', index=False)

In [44]:
# Diffs
def diff_numbers(df, feature):
    print(feature,', ', end='')
    for scope in ['tt','ts']:
        for typ in ['avg','form']:
            col1=f'tar_{feature}1_{scope}_{typ}'
            col2=f'opp_{feature}2_{scope}_{typ}'
            col_out1=f'{feature}1_{scope}_{typ}'
            df[col_out1]=(df[col1]+df[col2])/2
            col1=f'tar_{feature}2_{scope}_{typ}'
            col2=f'opp_{feature}1_{scope}_{typ}'
            col_out2=f'{feature}2_{scope}_{typ}'
            df[col_out2]=(df[col1]+df[col2])/2
            col_out=f'diff_{feature}_{scope}_{typ}'
            df[col_out]=df[col_out1]-df[col_out2]
    return df

def diff_non_numbers(df, feature):
    print(feature,', ', end='')
    for scope in ['tt','ts']:
        for typ in ['avg','form']:
            'tar_wx_tt_avg'
            col1=f'tar_{feature}_{scope}_{typ}'
            col2=f'opp_{feature}_{scope}_{typ}'
            col_out=f'{feature}_{scope}_{typ}'
            df[col_out]=(df[col1]+df[col2])/2
    return df

features=['w',  'ht', 'ft', 'graph', 'possession', 'shont', 'shofft', 'corners', 'offsides', 'fouls','cards', 'gksaves','precision']
features_non_numbers=['wx',  'psht', 'psft']

df=df_.copy()
for col in features:
    df=diff_numbers(df, col)

print('')
for col in features_non_numbers:
    df=diff_non_numbers(df, col)


w , ht , ft , graph , possession , shont , shofft , corners , offsides , fouls , cards , gksaves , precision , 
wx , psht , psft , 

In [46]:
df['diff_vote12']=df['vote1']-df_['vote2']
df['diff_elo']=df['elo1']-df_['elo2']
df['diff_op']=df['oddsprob_home']-df_['oddsprob_away']
df.to_csv('data/stats_generated.csv', index=False)

In [42]:
nulls=pd.DataFrame(df_.isna().sum(), columns=['n'])
nulls[nulls.n>100000]

,n
_diff_oddsprob_home_tt_form,106514
_diff_oddsprob_draw_tt_form,106514
_diff_oddsprob_away_tt_form,106514
_diff_drift_home_tt_form,106514
_diff_drift_away_tt_form,106514
_diff_drift_draw_tt_form,106514


# No all
df_1=df.copy()[COL_INF+COL_CUR]
df_1=df_1.merge(df_home[df_home.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
df_1=df_1.merge(df_away[df_away.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')
df_1.drop(columns=['tid_x','tid_y'], inplace=True)
cols_tar=[x for x in df_1.columns if '_th_' in x]
cols_opp=[x for x in df_1.columns if '_ta_' in x]
cols_diff=[x.replace('_th_','_diff_') for x in cols_tar]
df_1.reset_index(drop=True, inplace=True)
df_1=pd.concat([df_1,pd.DataFrame(df_1[cols_tar].values-df_1[cols_opp].values, columns=cols_diff)], axis=1)
df_1['diff_vote12']=df_1['vote1']-df_1['vote2']
df_1['diff_elo']=df_1['elo1']-df_1['elo2']
df_1['diff_op']=df_1['oddsprob_home']-df_1['oddsprob_away']

In [26]:
display(df_[df_.mid==id])

,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,odds_away,odds_draw,odds_home,country_id,round,ds,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,oddsprob_home,oddsprob_draw,oddsprob_away,drift_home,drift_away,drift_draw,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_ps_ht_tt_avg,tar_ps_ft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,tar_oddsprob_home_tt_avg,tar_oddsprob_draw_tt_avg,tar_oddsprob_away_tt_avg,tar_drift_home_tt_avg,tar_drift_away_tt_avg,tar_drift_draw_tt_avg,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_ps_ht_tt_avg,opp_ps_ft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,opp_oddsprob_home_tt_avg,opp_oddsprob_draw_tt_avg,opp_oddsprob_away_tt_avg,opp_drift_home_tt_avg,opp_drift_away_tt_avg,opp_drift_draw_tt_avg,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_ps_ht_ts_avg,tar_ps_ft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,tar_oddsprob_home_ts_avg,tar_oddsprob_draw_ts_avg,tar_oddsprob_away_ts_avg,tar_drift_home_ts_avg,tar_drift_away_ts_avg,tar_drift_draw_ts_avg,opp_w1_ts_avg,opp_wx_ts_avg,opp_w2_ts_avg,opp_ht1_ts_avg,opp_ht2_ts_avg,opp_ft1_ts_avg,opp_ft2_ts_avg,opp_ps_ht_ts_avg,opp_ps_ft_ts_avg,opp_vote1_ts_avg,opp_votex_ts_avg,opp_vote2_ts_avg,opp_elo1_ts_avg,opp_elo2_ts_avg,opp_oddsprob_home_ts_avg,opp_oddsprob_draw_ts_avg,opp_oddsprob_away_ts_avg,opp_drift_home_ts_avg,opp_drift_away_ts_avg,opp_drift_draw_ts_avg,_diff_w1_tt_avg,_diff_wx_tt_avg,_diff_w2_tt_avg,_diff_ht1_tt_avg,_diff_ht2_tt_avg,_diff_ft1_tt_avg,_diff_ft2_tt_avg,_diff_ps_ht_tt_avg,_diff_ps_ft_tt_avg,_diff_vote1_tt_avg,_diff_votex_tt_avg,_diff_vote2_tt_avg,_diff_elo1_tt_avg,_diff_elo2_tt_avg,_diff_oddsprob_home_tt_avg,_diff_oddsprob_draw_tt_avg,_diff_oddsprob_away_tt_avg,_diff_drift_home_tt_avg,_diff_drift_away_tt_avg,_diff_drift_draw_tt_avg,_diff_w1_ts_avg,_diff_wx_ts_avg,_diff_w2_ts_avg,_diff_ht1_ts_avg,_diff_ht2_ts_avg,_diff_ft1_ts_avg,_diff_ft2_ts_avg,_diff_ps_ht_ts_avg,_diff_ps_ft_ts_avg,_diff_vote1_ts_avg,_diff_votex_ts_avg,_diff_vote2_ts_avg,_diff_elo1_ts_avg,_diff_elo2_ts_avg,_diff_oddsprob_home_ts_avg,_diff_oddsprob_draw_ts_avg,_diff_oddsprob_away_ts_avg,_diff_drift_home_ts_avg,_diff_drift_away_ts_avg,_diff_drift_draw_ts_avg,diff_vote12,diff_elo,diff_op
67788,romania,liga-i,9270007,12,2021-01-10 17:00:00+00:00,fc viitorul constanta,fc hermannstadt,594,1499,1,0,0,2.0,1.0,home,NaN,NaN,NaN,34,12,2021-01-10 17:00:00+00:00,2021-01-10,18,14,0.623808,0.307603,0.068589,0,1352.687866,1285.478027,NaN,NaN,NaN,NaN,NaN,NaN,0.418994,0.245810,0.335196,0.597765,0.564246,1.441341,1.223464,0.033520,0.217877,0.421861,0.246587,0.331553,1405.488915,1350.262910,NaN,NaN,NaN,NaN,NaN,NaN,0.273684,0.315789,0.410526,0.431579,0.515789,1.021053,1.357895,-0.084211,-0.336842,0.262723,0.286901,0.450376,1238.260056,1280.394616,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,0.204545,0.295455,0.727273,0.477273,1.715909,1.056818,0.250000,0.659091,0.517677,0.237200,0.245123,1413.419866,1351.429943,NaN,NaN,NaN,NaN,NaN,NaN,0.191489,0.340426,0.468085,0.340426,0.702128,0.936170,1.574468,-0.361702,-0.638298,0.144103,0.278918,0.576980,1251.697931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.14531,-0.069979,-0.075331,0.166186,0.048456,0.420288,-0.134431,0.11773,0.554719,0.159138,-0.040315,-0.118823,167.228859,69.868294,NaN,NaN,NaN,NaN,NaN,NaN,0.308511,-0.13588,-0.172631,0.386847,-0.224855,0.779739,-0.51765,0.611702,1.297389,0.373574,-0.041718,-0.331856,161.721936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.55522,67.209839,NaN
135578,romania,liga-i,9270007,12,2021-01-10 17:00:00+00:00,fc hermannstadt,fc viitorul constanta,1499,594,0,0,1,1.0,2.0,home,NaN,NaN,NaN,34,12,2021-01-10 17:00:00+00:00,2021-01-10,14,18,0.068589,0.307603,0.623808,0,1285.478027,1352.687866,NaN,NaN,NaN,NaN,NaN,NaN,0.273684,0.31578

In [67]:
df_home[df_home.mid==8736289]

,ds,mid,tid,w1_th_avg,wx_th_avg,w2_th_avg,ht1_th_avg,ht2_th_avg,ft1_th_avg,ft2_th_avg,ps_ht_th_avg,ps_ft_th_avg,vote1_th_avg,votex_th_avg,vote2_th_avg,elo1_th_avg,elo2_th_avg,oddsprob_home_th_avg,oddsprob_draw_th_avg,oddsprob_away_th_avg,drift_home_th_avg,drift_away_th_avg,drift_draw_th_avg
67789,2021-01-10 15:00:00+00:00,8736289,685,0.433333,0.133333,0.433333,0.6,0.866667,1.3,1.566667,-0.266667,-0.266667,0.384568,0.214804,0.400628,1030.232308,1012.008004,0.162159,0.124672,0.213169,-0.009556,0.01476,0.011214


In [68]:
df_away[df_away.mid==8736289]

,ds,mid,tid,w1_ta_avg,wx_ta_avg,w2_ta_avg,ht1_ta_avg,ht2_ta_avg,ft1_ta_avg,ft2_ta_avg,ps_ht_ta_avg,ps_ft_ta_avg,vote1_ta_avg,votex_ta_avg,vote2_ta_avg,elo1_ta_avg,elo2_ta_avg,oddsprob_home_ta_avg,oddsprob_draw_ta_avg,oddsprob_away_ta_avg,drift_home_ta_avg,drift_away_ta_avg,drift_draw_ta_avg
67789,2021-01-10 15:00:00+00:00,8736289,306,0.27027,0.243243,0.486486,0.558559,0.918919,1.216216,1.774775,-0.36036,-0.558559,0.238312,0.197249,0.564439,1024.520863,1114.002228,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_.to_csv('data/stats_generated.csv', index=False)

In [75]:
df=pd.read_csv('data/stats_generated.csv', index_col=None)
df=df.dropna()

In [82]:
COL_CUR=['side', 'country_id', 'round', 'ds', 'de', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2']
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft']
COL_CAT=['country_id','form1', 'form2']
COL_BIN=['side']

COL_INF=['country', 'liga', 'mid', 'round', 'ds', 't1', 't2','tid1', 'tid2', 'w1', 'wx', 'w2',  'ft1', 'ft2','winner']

In [88]:
scaler=MinMaxScaler()
nums=scaler.fit_transform(df[df.columns[25:]].values)
nums_df=pd.DataFrame(nums, columns=df.columns[25:])
df.reset_index(drop=True, inplace=True)
df=pd.concat([df[df.columns[:25]],nums_df], axis=1)

,pop_r,elo1,elo2,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_ps_ht_tt_avg,tar_ps_ft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_ps_ht_tt_avg,opp_ps_ft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_ps_ht_ts_avg,tar_ps_ft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,opp_w1_ts_avg,opp_wx_ts_avg,opp_w2_ts_avg,opp_ht1_ts_avg,opp_ht2_ts_avg,opp_ft1_ts_avg,opp_ft2_ts_avg,opp_ps_ht_ts_avg,opp_ps_ft_ts_avg,opp_vote1_ts_avg,opp_votex_ts_avg,opp_vote2_ts_avg,opp_elo1_ts_avg,opp_elo2_ts_avg,diff_w1_tt_avg,diff_wx_tt_avg,diff_w2_tt_avg,diff_ht1_tt_avg,diff_ht2_tt_avg,diff_ft1_tt_avg,diff_ft2_tt_avg,diff_ps_ht_tt_avg,diff_ps_ft_tt_avg,diff_vote1_tt_avg,diff_votex_tt_avg,diff_vote2_tt_avg,diff_elo1_tt_avg,diff_elo2_tt_avg,diff_w1_ts_avg,diff_wx_ts_avg,diff_w2_ts_avg,diff_ht1_ts_avg,diff_ht2_ts_avg,diff_ft1_ts_avg,diff_ft2_ts_avg,diff_ps_ht_ts_avg,diff_ps_ft_ts_avg,diff_vote1_ts_avg,diff_votex_ts_avg,diff_vote2_ts_avg,diff_elo1_ts_avg,diff_elo2_ts_avg,diff_vote12,diff_elo
0,1.00,0.472463,0.472463,1.000000,0.000000,0.000000,0.571429,0.000000,0.500000,0.000000,0.823529,0.782609,0.858299,0.162021,0.104519,1.000000,0.635697,0.708861,0.189873,0.101266,0.211573,0.041772,0.292194,0.094937,0.650782,0.624656,0.708244,0.362246,0.156602,0.836029,0.678962,1.000000,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.777778,0.785714,0.861679,0.162021,0.097464,0.995360,0.635697,0.000000,0.000000,1.000000,0.000000,0.400000,0.166667,0.375000,0.333333,0.428571,0.332264,0.305874,0.560808,0.892168,0.931487,0.645570,0.405063,0.449367,0.709916,0.476503,0.624684,0.442814,0.640587,0.618465,0.598509,0.322713,0.469866,0.647809,0.455834,1.000000,0.500000,0.000000,0.750000,0.300000,0.666667,0.306452,0.833333,0.750000,0.785620,0.368704,0.246279,0.596018,0.186188,0.725685,0.500000
1,0.75,0.628003,0.657831,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.588235,0.695652,0.085197,0.391887,0.799607,0.464370,0.471198,0.442748,0.282443,0.274809,0.196292,0.103817,0.258270,0.213740,0.607993,0.544972,0.475809,0.485643,0.336685,0.648486,0.676920,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.555556,0.714286,0.107016,0.391887,0.745632,0.470917,0.471198,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.555556,0.714286,0.122613,0.410600,0.720589,0.470917,0.471198,0.778626,0.358779,0.362595,0.385496,0.441603,0.545038,0.371251,0.483921,0.613010,0.243569,0.416984,0.767833,0.334032,0.289995,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.491586,0.482920,0.513713,0.500000,0.500000,0.626219,0.472435
2,1.00,0.811409,0.523643,1.000000,0.000000,0.000000,0.857143,0.000000,1.000000,0.166667,0.941176,0.956522,0.874874,0.211592,0.060606,0.464370,0.471198,0.408333,0.266667,0.325000,0.180952,0.120000,0.229167,0.225000,0.592157,0.523913,0.424758,0.522580,0.371003,0.468630,0.516784,1.000000,0.000000,0.000000,0.750000,0.000000,1.000000,0.125000,0.888889,0.928571,0.877859,0.211592,0.056515,0.470917,0.471198,0.000000,0.000000,1.000000,0.000000,0.600000,0.166667,0.750000,0.222222,0.214286,0.065924,0.211592,0.876629,0.470917,0.471198,0.795833,0.366667,0.337500,0.894444,0.432500,0.962500,0.464862,0.784043,0.824457,0.795494,0.224639,0.320414,0.496160,0.453465,1.000000,0.500000,0.000000,0.875000,0.200000,0.916667,0.177419,1.000000,1.000000,0.938040,0.500000,0.050917,0.500000,0.500000,0.878608,0.765927
3,1.00,0.756613,0.697733,1.000000,0.000000,0.000000,0.571429,0.000000,0.666667,0.000000,0.823529,0.869565,0.858765,0.293601,0.033987,0.464370,0.471198,0.379032,0.274194,0.346774,0.168203,0.087097,0.206989,0.181452,0.606262,

In [106]:
df_info=df[COL_INF]
labels=df[['w1', 'wx', 'w2']].values

encoder = OneHotEncoder()
countries=encoder.fit_transform(df[['country_id']]).toarray()
encoder = OneHotEncoder()
form1=encoder.fit_transform(df[['form1']]).toarray()
encoder = OneHotEncoder()
form2=encoder.fit_transform(df[['form2']]).toarray()
side=df[['side']].values

data=np.hstack([nums,countries,form1,form2,side])

In [108]:
data.shape,labels.shape

((130140, 187), (130140, 3))

(130140, 187)

In [81]:
COL_NUM=['pop_r', 'elo1', 'elo2']

,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,side,country_id,round,ds,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_ps_ht_tt_avg,tar_ps_ft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_ps_ht_tt_avg,opp_ps_ft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_ps_ht_ts_avg,tar_ps_ft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,opp_w1_ts_avg,opp_wx_ts_avg,opp_w2_ts_avg,opp_ht1_ts_avg,opp_ht2_ts_avg,opp_ft1_ts_avg,opp_ft2_ts_avg,opp_ps_ht_ts_avg,opp_ps_ft_ts_avg,opp_vote1_ts_avg,opp_votex_ts_avg,opp_vote2_ts_avg,opp_elo1_ts_avg,opp_elo2_ts_avg,diff_w1_tt_avg,diff_wx_tt_avg,diff_w2_tt_avg,diff_ht1_tt_avg,diff_ht2_tt_avg,diff_ft1_tt_avg,diff_ft2_tt_avg,diff_ps_ht_tt_avg,diff_ps_ft_tt_avg,diff_vote1_tt_avg,diff_votex_tt_avg,diff_vote2_tt_avg,diff_elo1_tt_avg,diff_elo2_tt_avg,diff_w1_ts_avg,diff_wx_ts_avg,diff_w2_ts_avg,diff_ht1_ts_avg,diff_ht2_ts_avg,diff_ft1_ts_avg,diff_ft2_ts_avg,diff_ps_ht_ts_avg,diff_ps_ft_ts_avg,diff_vote1_ts_avg,diff_votex_ts_avg,diff_vote2_ts_avg,diff_elo1_ts_avg,diff_elo2_ts_avg,diff_vote12,diff_elo
73,spain,copa-del-rey,6570261,4,2015-01-15 19:00:00+00:00,real madrid,atletico madrid,9,134,0,1,0,2.0,2.0,draw,1,42,4,2015-01-15 19:00:00+00:00,2015-01-15,18,20,0.671697,0.094438,0.233865,4,1512.538486,1512.538486,1.000000,0.000000,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,3.000000,0.800165,0.070749,0.129086,2071.864258,1687.874878,0.708861,0.189873,0.101266,0.740506,0.208861,1.753165,0.569620,0.531646,1.183544,0.670008,0.158180,0.171812,1900.639220,1733.990657,1.000000,0.000000,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,3.000000,0.800165,0.070749,0.129086,2071.864258,1687.874878,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,3.000000,-2.000000,-2.000000,0.329731,0.133564,0.536705,1961.808838,2003.151367,0.291139,-0.189873,-0.101266,1.259494,-0.208861,1.246835,-0.569620,1.468354,1.816456,0.130158,-0.087431,-0.042727,1.712250e+02,-4.611578e+01,1.000000,0.000000,-1.000000,2.000000,-2.000000,2.000000,-3.000000,4.000000,5.000000,0.470435,-0.062816,-0.407619,110.055420,-315.276489,0.437832,0.000000
115,spain,laliga,5764480,19,2015-01-17 21:00:00+00:00,espanyol,celta vigo,59,17,1,0,0,1.0,0.0,home,1,42,19,2015-01-17 21:00:00+00:00,2015-01-17,20,7,0.500000,0.244866,0.255134,3,1686.446167,1719.797363,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.129576,0.171123,0.699301,1512.538486,1512.538486,0.442748,0.282443,0.274809,0.687023,0.519084,1.549618,1.282443,0.167939,0.267176,0.468394,0.212063,0.319543,1704.799281,1731.814017,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.129576,0.171123,0.699301,1512.538486,1512.538486,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.143436,0.179294,0.677270,1512.538486,1512.538486,0.557252,-0.282443,-0.274809,-0.687023,-0.519084,0.450382,-1.282443,-0.167939,1.732824,-0.338818,-0.040940,0.379758,-1.922608e+02,-2.192755e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.013859,-0.008171,0.022031,0.000000,0.000000,0.244866,-33.351196
143,italy,serie-a,5786106,19,2015-01-18 19:45:00+00:00,juventus,hellas verona,73,33,1,0,0,4.0,0.0,home,1,24,19,2015-01-18 19:45:00+00:00,2015-01-18,16,23,0.826446,0.081612,0.091942,4,1891.511353,1569.762085,1.000000,0.000000,0.000000,3.000000,0.000000,6.000000,1.000000,3.000000,5.000000,0.814543,0.092395,0.093062,1512.538486,1512.538486,0.408333,0.266667,0.325000,0.633333,0.600000,1.375000,1.350000,0.033333,0.025000,0.424112,0.2281

In [43]:
df_[df_['mid']==9197411]

,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,side,country_id,round,ds,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,tid_x,home_w1_tt_avg,home_wx_tt_avg,home_w2_tt_avg,home_ht1_tt_avg,home_ht2_tt_avg,home_ft1_tt_avg,home_ft2_tt_avg,home_ps_ht_tt_avg,home_ps_ft_tt_avg,home_vote1_tt_avg,home_votex_tt_avg,home_vote2_tt_avg,home_elo1_tt_avg,home_elo2_tt_avg,tid_y,w1_th_avg,wx_th_avg,w2_th_avg,ht1_th_avg,ht2_th_avg,ft1_th_avg,ft2_th_avg,ps_ht_th_avg,ps_ft_th_avg,vote1_th_avg,votex_th_avg,vote2_th_avg,elo1_th_avg,elo2_th_avg,tid_x,away_w1_tt_avg,away_wx_tt_avg,away_w2_tt_avg,away_ht1_tt_avg,away_ht2_tt_avg,away_ft1_tt_avg,away_ft2_tt_avg,away_ps_ht_tt_avg,away_ps_ft_tt_avg,away_vote1_tt_avg,away_votex_tt_avg,away_vote2_tt_avg,away_elo1_tt_avg,away_elo2_tt_avg,tid_y,w1_ta_avg,wx_ta_avg,w2_ta_avg,ht1_ta_avg,ht2_ta_avg,ft1_ta_avg,ft2_ta_avg,ps_ht_ta_avg,ps_ft_ta_avg,vote1_ta_avg,votex_ta_avg,vote2_ta_avg,elo1_ta_avg,elo2_ta_avg
67450,greece,super-league,9197411,11,2020-12-05 17:30:00+00:00,olympiacos,volos nfc,319,1653,1,0,0,4.0,1.0,home,1,21,11,2020-12-05 17:30:00+00:00,2020-12-05,14,14,0.776807,0.169688,0.053505,2,1686.436646,1252.658447,319,0.750000,0.132812,0.117188,0.9375,0.234375,2.3125,0.609375,0.703125,1.703125,0.660547,0.160972,0.178481,1664.798659,1461.905214,319.0,0.75,0.132812,0.117188,0.9375,0.234375,2.3125,0.609375,0.703125,1.703125,0.660547,0.160972,0.178481,1664.798659,1461.905214,1653,0.270833,0.291667,0.437500,0.3125,0.666667,0.9375,1.500000,-0.354167,-0.562500,0.232241,0.277189,0.490570,1296.581658,1412.341579,1653.0,0.208333,0.291667,0.5,0.416667,0.666667,1.0,1.791667,-0.25,-0.791667,0.122681,0.26322,0.614099,1300.265222,1419.646438
135240,greece,super-league,9197411,11,2020-12-05 17:30:00+00:00,volos nfc,olympiacos,1653,319,0,0,1,1.0,4.0,home,0,21,11,2020-12-05 17:30:00+00:00,2020-12-05,14,14,0.053505,0.169688,0.776807,2,1252.658447,1686.436646,1653,0.270833,0.291667,0.437500,0.3125,0.666667,0.9375,1.500000,-0.354167,-0.562500,0.232241,0.277189,0.490570,1296.581658,1412.341579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319,0.750000,0.132812,0.117188,0.9375,0.234375,2.3125,0.609375,0.703125,1.703125,0.660547,0.160972,0.178481,1664.798659,1461.905214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
